In [1]:
import joblib
import pandas as pd
import numpy as np
from tqdm import tqdm 
from glob import glob
from imp import reload
import copy as cp
from utils import util
reload(util)

C:\Users\zhou\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


<module 'utils.util' from 'D:\\pycharm_project\\optiver-realized-volatility-prediction\\src\\utils\\util.py'>

In [2]:
import os
path_lst = glob('../data/book_train.parquet/*')
stock_lst = [os.path.basename(path).split('=')[-1] for path in path_lst]

In [3]:
print(len(stock_lst))

112


In [4]:
temp = util.gen_data_train(0)

C:\Users\zhou\AppData\Roaming\Python\Python38\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\zhou\AppData\Roaming\Python\Python38\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


isBS B
isBS S
isBS_big supB
isBS_big supS
isBS_big midBS
isoversize50 up50
isoversize50 down50
isoversize25 up25
isoversize25 down25
isoversize75 up75
isoversize75 down75


C:\Users\zhou\AppData\Roaming\Python\Python38\site-packages\pandas\core\reshape\merge.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  return op.get_result()


In [5]:
data_type = 'train'
# fe_df, stock_df = util.gen_data_multi(stock_lst, data_type)
fe_df = util.gen_data_multi(stock_lst, data_type)

100%|██████████| 112/112 [19:43<00:00, 10.57s/it]


In [ ]:
# stock_df.to_pickle('../data/train_stock_df.pkl')

In [6]:
train = pd.read_csv('../data/train.csv')

In [ ]:
train.head(5)

In [7]:

# fe_df = fe_df.merge(
#         stock_df
#         , how='left'
#         , on='stock_id'
#     ).merge(
#         train
#         , how='left'
#         , on=['stock_id', 'time_id']
#     ).replace([np.inf, -np.inf], np.nan).fillna(method='ffill')

fe_df = fe_df.merge(
        train
        , how='left'
        , on=['stock_id', 'time_id']
    ).replace([np.inf, -np.inf], np.nan).fillna(method='ffill')

In [5]:
# for name in tqdm(name_lst):
#     ret = util.gen_data(name)
#     ret.to_csv('../data/20210731/{}.csv'.format())

In [8]:
import  lightgbm  as lgb

# LightGBM parameters
params = {
    'n_estimators': 10000,
    'objective': 'rmse',
    'boosting_type': 'gbdt',
    'max_depth': -1,
    'learning_rate': 0.01,
    'subsample': 0.72,
    'subsample_freq': 4,
    'feature_fraction': 0.8,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'seed': 46,
    'early_stopping_rounds': 300,
    'verbose': -1
} 
from sklearn import model_selection, logger

In [9]:
def RMSPEMetric(XGBoost=False):
    def RMSPE(yhat, dtrain, XGBoost=XGBoost):

        y = dtrain.get_label()
        elements = ((y - yhat) / y)**2
        if XGBoost:
            return 'RMSPE', float(np.sqrt(np.sum(elements) / len(y)))
        else:
            return 'RMSPE', float(np.sqrt(np.sum(elements) / len(y))), False

    return RMSPE

In [10]:
# data = fe_df
label = fe_df['target']
features = fe_df.columns.difference(['time_id','target']).tolist()

In [12]:
cats = ['stock_id', ]
# X_train = data_.reset_index(drop=True)
X_train = fe_df[features].reset_index(drop=True)

y_train = label
# y_train = pd.DataFrame(label_)
models = []
# oof_df = X_train[['time_id', 'stock_id']].copy()
oof_df = fe_df[['time_id', 'stock_id']].copy()

oof_df['target'] = y_train
oof_df['pred'] = np.nan

cv = model_selection.KFold(n_splits=10,
                            shuffle=True,
                            random_state=666)

kf = cv.split(X_train, y_train)

fi_df = pd.DataFrame()
fi_df['features'] = features
fi_df['importance'] = 0

for fold_id, (train_index, valid_index) in tqdm(enumerate(kf)):
    # split
    X_tr = X_train.loc[train_index, features]
    X_val = X_train.loc[valid_index, features]
    y_tr = y_train.loc[train_index].values.reshape(-1)
    y_val = y_train.loc[valid_index].values.reshape(-1)

    # model (note inverse weighting)
    train_set = lgb.Dataset(X_tr,
                            y_tr,
                            categorical_feature=cats,
                            weight=1 / np.power(y_tr, 2))
    val_set = lgb.Dataset(X_val,
                          y_val,
                          categorical_feature=cats,
                          weight=1 / np.power(y_val, 2))
    model = lgb.train(params,
                      train_set,
                      valid_sets=[train_set, val_set],
                      feval=RMSPEMetric(),
                      verbose_eval=250)

    # feature importance
    fi_df[f'importance_fold{fold_id}'] = model.feature_importance(
        importance_type="gain")
    fi_df['importance'] += fi_df[f'importance_fold{fold_id}'].values

    # save model
    joblib.dump(model, f'model_fold{fold_id}.pkl')
    logger.debug('model saved!')

0it [00:00, ?it/s]C:\Users\zhou\AppData\Roaming\Python\Python38\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\Users\zhou\AppData\Roaming\Python\Python38\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\zhou\AppData\Roaming\Python\Python38\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\zhou\AppData\Roaming\Python\Python38\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters

Training until validation scores don't improve for 300 rounds
[250]	training's l2: 2.7563e-07	training's RMSPE: 0.242859	valid_1's l2: 2.81026e-07	valid_1's RMSPE: 0.246434
[500]	training's l2: 2.49816e-07	training's RMSPE: 0.231207	valid_1's l2: 2.57671e-07	valid_1's RMSPE: 0.235972
[750]	training's l2: 2.41922e-07	training's RMSPE: 0.227524	valid_1's l2: 2.54012e-07	valid_1's RMSPE: 0.234291
[1000]	training's l2: 2.36971e-07	training's RMSPE: 0.225185	valid_1's l2: 2.53175e-07	valid_1's RMSPE: 0.233904
[1250]	training's l2: 2.33245e-07	training's RMSPE: 0.223407	valid_1's l2: 2.52427e-07	valid_1's RMSPE: 0.233559
[1500]	training's l2: 2.30188e-07	training's RMSPE: 0.221938	valid_1's l2: 2.52451e-07	valid_1's RMSPE: 0.233569


1it [01:18, 78.31s/it]

Early stopping, best iteration is:
[1345]	training's l2: 2.32017e-07	training's RMSPE: 0.222818	valid_1's l2: 2.52254e-07	valid_1's RMSPE: 0.233478
Training until validation scores don't improve for 300 rounds
[250]	training's l2: 2.74743e-07	training's RMSPE: 0.242769	valid_1's l2: 2.88874e-07	valid_1's RMSPE: 0.247068
[500]	training's l2: 2.49331e-07	training's RMSPE: 0.231269	valid_1's l2: 2.66252e-07	valid_1's RMSPE: 0.237196
[750]	training's l2: 2.41532e-07	training's RMSPE: 0.227623	valid_1's l2: 2.62081e-07	valid_1's RMSPE: 0.235331
[1000]	training's l2: 2.36538e-07	training's RMSPE: 0.225258	valid_1's l2: 2.60255e-07	valid_1's RMSPE: 0.23451
[1250]	training's l2: 2.32854e-07	training's RMSPE: 0.223497	valid_1's l2: 2.59965e-07	valid_1's RMSPE: 0.234379


2it [02:30, 74.60s/it]

...

8it [17:35, 151.17s/it]

Training until validation scores don't improve for 300 rounds
[250]	training's l2: 2.75855e-07	training's RMSPE: 0.242944	valid_1's l2: 2.79835e-07	valid_1's RMSPE: 0.246035
[500]	training's l2: 2.49989e-07	training's RMSPE: 0.231274	valid_1's l2: 2.5627e-07	valid_1's RMSPE: 0.235448
[750]	training's l2: 2.4202e-07	training's RMSPE: 0.227558	valid_1's l2: 2.52812e-07	valid_1's RMSPE: 0.233854
[1000]	training's l2: 2.3708e-07	training's RMSPE: 0.225223	valid_1's l2: 2.52109e-07	valid_1's RMSPE: 0.233529
[1250]	training's l2: 2.33397e-07	training's RMSPE: 0.223467	valid_1's l2: 2.5229e-07	valid_1's RMSPE: 0.233612


9it [18:50, 127.48s/it]

Early stopping, best iteration is:
[1173]	training's l2: 2.34446e-07	training's RMSPE: 0.223969	valid_1's l2: 2.51736e-07	valid_1's RMSPE: 0.233356
Training until validation scores don't improve for 300 rounds
[250]	training's l2: 2.75059e-07	training's RMSPE: 0.242839	valid_1's l2: 2.86155e-07	valid_1's RMSPE: 0.246543
[500]	training's l2: 2.4927e-07	training's RMSPE: 0.231175	valid_1's l2: 2.61923e-07	valid_1's RMSPE: 0.235873
[750]	training's l2: 2.4152e-07	training's RMSPE: 0.227553	valid_1's l2: 2.57349e-07	valid_1's RMSPE: 0.233805
[1000]	training's l2: 2.36568e-07	training's RMSPE: 0.225208	valid_1's l2: 2.54799e-07	valid_1's RMSPE: 0.232643
[1250]	training's l2: 2.32835e-07	training's RMSPE: 0.223424	valid_1's l2: 2.5366e-07	valid_1's RMSPE: 0.232123
[1500]	training's l2: 2.29777e-07	training's RMSPE: 0.221952	valid_1's l2: 2.52929e-07	valid_1's RMSPE: 0.231788
[1750]	training's l2: 2.27167e-07	training's RMSPE: 0.220688	valid_1's l2: 2.52385e-07	valid_1's RMSPE: 0.231539
[2000

10it [21:32, 129.29s/it]


In [18]:
def calc_rollingstats(rolling_x, roll_name):
    #统计量
    roll_autocorr =  rolling_x.groupby("time_id")[[roll_name,"xpre"]].corr()
    roll_autocorr .reset_index(inplace=True)
    roll_autocorr = roll_autocorr.groupby("time_id").head(1)
    roll_autocorr.index = roll_autocorr["time_id"]
    del roll_autocorr["time_id"]
    roll_autocorr = pd.DataFrame({roll_name+"_autocorr": roll_autocorr["xpre"]})
    
    roll_mean = pd.DataFrame({roll_name+"_mean": rolling_x.groupby("time_id")[roll_name].mean()})
    roll_std = pd.DataFrame({roll_name+"_std": rolling_x.groupby("time_id")[roll_name].std()})
    roll_skew = pd.DataFrame({roll_name+"_skew": rolling_x.groupby("time_id")[roll_name].skew()})
    
    data_merge = pd.merge(roll_mean, roll_std, left_index=True, right_index=True,how="inner")
    data_merge = pd.merge(data_merge , roll_skew, left_index=True, right_index=True,how="inner")
    data_merge = pd.merge(data_merge , roll_autocorr, left_index=True, right_index=True,how="inner")
    return data_merge

def make_candle(df_data, price_name, vol_name, amt_name):
    
    df_data["pre"] = df_data.groupby("time_id")[price_name].shift(1)
    df_data["ret"] = df_data[price_name] / df_data["pre"] - 1
    df_data["absret"] = abs(df_data["ret"] )
    df_retsum = pd.DataFrame({"retsum":df_data.groupby("time_id")["ret"].sum()})
    df_absretsum = pd.DataFrame({"absretsum":df_data.groupby("time_id")["absret"].sum()})
    
    df_data["absobv"] = df_data["absret"] * df_data[vol_name]
    df_obvabs = pd.DataFrame({"xf4_abs":df_data.groupby("time_id")["absobv"].sum()})
    
    df_data["obv"] = df_data["ret"] * df_data[vol_name]
    df_obv = pd.DataFrame({"xf4":df_data.groupby("time_id")["obv"].sum()})
    
    df_amt =  pd.DataFrame({amt_name + "sum": df_data.groupby("time_id")[amt_name].sum()})
    df_vol =  pd.DataFrame({vol_name + "sum": df_data.groupby("time_id")[vol_name].sum()})
    
    df_mean = pd.DataFrame({price_name + "mean": df_data.groupby("time_id")[price_name].mean()})
    df_high = pd.DataFrame({price_name + "high": df_data.groupby("time_id")[price_name].max()})
    df_low = pd.DataFrame({price_name + "low": df_data.groupby("time_id")[price_name].min()})
    
    df_open = df_data.groupby("time_id").head(1)
    df_open.index = df_open ["time_id"]
    df_open = pd.DataFrame({price_name + "open": df_open[price_name]})
    
    df_close = df_data.groupby("time_id").tail(1)
    df_close.index = df_close["time_id"]
    df_close = pd.DataFrame({price_name + "close": df_close[price_name]})
    
    df_candle = pd.merge(df_high, df_low, left_index=True, right_index=True,how="inner")
    df_candle = pd.merge(df_candle, df_mean, left_index=True, right_index=True,how="inner")
    df_candle = pd.merge(df_candle, df_open, left_index=True, right_index=True,how="inner")
    df_candle = pd.merge(df_candle, df_close, left_index=True, right_index=True,how="inner")
    df_candle = pd.merge(df_candle, df_vol, left_index=True, right_index=True,how="inner")
    df_candle = pd.merge(df_candle, df_amt, left_index=True, right_index=True,how="inner")    
    df_candle = pd.merge(df_candle, df_retsum, left_index=True, right_index=True,how="inner")    
    df_candle = pd.merge(df_candle, df_absretsum, left_index=True, right_index=True,how="inner")    
    df_candle = pd.merge(df_candle, df_obvabs, left_index=True, right_index=True,how="inner")    
    df_candle = pd.merge(df_candle, df_obv, left_index=True, right_index=True,how="inner")    
    
    return df_candle 

def cal_candlefactor(df_candle, price_name, vol_name, amt_name):
    f_name = price_name + "candle"
    #f1:illiq
    df_candle[f_name + "f1"] = (2 * (df_candle[price_name + "high"] - df_candle[price_name + "low"]) 
                    - abs(df_candle[price_name + "open"] - df_candle[price_name + "close"]))/df_candle[amt_name + "sum"]
    #f2 strength
    df_candle[f_name + "f2"] = df_candle["retsum"]/df_candle["absretsum"]
    #f3:ad
    df_candle[f_name + "f3"] =  (2 *df_candle[price_name + "close"] - df_candle[price_name + "low"]\
                    - df_candle[price_name + "high"] )/(df_candle[price_name + "high"] - df_candle[price_name + "low"]) \
                    * df_candle[vol_name + "sum"]
    #f3: obv
    df_candle[f_name + "f41"] =  df_candle["xf4"]/df_candle[vol_name + "sum"]
    df_candle[f_name + "f42"] =  df_candle["xf4_abs"]/df_candle[vol_name + "sum"]
    return df_candle

# def calc_rollingstats_realvol(rolling_x, roll_name):
#     #统计量
#     vol_new =  pd.DataFrame({roll_name+"_autocorr": rolling_x.groupby('time_id')[roll_name].apply(
#                                     lambda x: np.sqrt(np.sum(np.log(x).diff()**2)))})
#     data_merge = pd.merge(data_merge , vol_new, left_index=True, right_index=True,how="inner")
#     return data_merge

def calculate_features2(book_df, trade_df):
    """
    df: book_train data for each stock_id
    """
    #calculate price for features
    
    book_df['wap'] = (book_df['bid_price1'] * book_df['ask_size1'] + book_df['ask_price1'] * 
                       book_df['bid_size1']) / (book_df['bid_size1']+ book_df['ask_size1'])
    
    book_df["vol_ab"] = book_df['bid_size1']+ book_df['ask_size1']
    book_df["amt_ab"] = book_df['bid_price1'] * book_df['ask_size1'] + book_df['ask_price1'] * book_df['bid_size1']
    
    book_df["amt_a"] = book_df['ask_price1'] * book_df['ask_size1'] 
    book_df["amt_b"] = book_df['bid_price1'] * book_df['bid_size1'] 
    
    trade_df["amt"] = trade_df["price"] * trade_df["size"]
    
    #flag filter
    book_df["wap_pre"] = book_df.groupby("time_id")['wap'].shift(1)
    book_df["bid_ppre"] = book_df.groupby("time_id")['bid_price1'].shift(1)
    book_df["ask_ppre"] = book_df.groupby("time_id")['ask_price1'].shift(1)
    
    
    book_df["isBS"] = np.where(book_df["wap"]>book_df["wap_pre"], "B",
                       np.where(book_df["wap"]<book_df["wap_pre"], "S",np.nan))
    book_df["isBS_big"] = np.where(book_df["wap"]>book_df["ask_ppre"], "supB",
                       np.where(book_df["wap"]<book_df["bid_ppre"], "supS",
                        np.where(pd.notnull(book_df["wap"]),"midBS",np.nan)))
    
    ordersize50 = pd.DataFrame({"ordersize50":book_df.groupby("time_id")["amt_ab"].apply(lambda x :np.nanmedian(x))})
    ordersize50.reset_index(inplace=True)
    
    ordersize25 = pd.DataFrame({"ordersize25": book_df.groupby("time_id")["amt_ab"].apply(lambda x :np.nanpercentile(x,75))})
    ordersize25.reset_index(inplace=True)
    
    ordersize75 = pd.DataFrame({"ordersize75": book_df.groupby("time_id")["amt_ab"].apply(lambda x :np.nanpercentile(x, 25))}) 
    ordersize75.reset_index(inplace=True)
    book_df1 = pd.merge(book_df, ordersize50, on="time_id", how="left")
    book_df1 = pd.merge(book_df1, ordersize25, on="time_id", how="left")
    book_df1 = pd.merge(book_df1, ordersize75, on="time_id", how="left")
    book_df1.loc[:,"isoversize50"] = np.where(book_df1["amt_ab"]>book_df1["ordersize50"],"up50",
                                     np.where(book_df1["amt_ab"]<=book_df1["ordersize50"],"down50",np.nan))
    
    
    book_df1.loc[:,"isoversize75"] = np.where(book_df1["amt_ab"]>book_df1["ordersize75"],"up75",
                                     np.where(book_df1["amt_ab"]<=book_df1["ordersize75"],"down75",np.nan))
    book_df1.loc[:,"isoversize25"] = np.where(book_df1["amt_ab"]>book_df1["ordersize25"],"up25",
                                     np.where(book_df1["amt_ab"]<=book_df1["ordersize25"],"down25",np.nan))
    
     #不同波动率
    #calculate historical volatility
    vol = book_df1.groupby('time_id')['wap'].apply(lambda x: np.sqrt(np.sum(np.log(x).diff()**2)))
    vol_df = pd.DataFrame(vol)
    vol_df.rename(columns={'wap': 'vol_orig'}, inplace=True)
    data_merge_all = vol_df
        
    #修改波动率：
#    rolling波动率均值，标准差，偏度，自相关
    #新指标,新指标均值，标准差，偏度，自相关

#    roll_name0 = "roll_std" 
#    roll_window = 10
    #BS FLAG
    flagname = "B"
    filtername = "isBS"
    for  filtername, flagname in [["isBS","B"],["isBS","S"],
                                  ["isBS_big","supB"],["isBS_big","supS"],["isBS_big","midBS"],
                                  ["isoversize50","up50"],["isoversize50","down50"],
                                  ["isoversize25","up25"],["isoversize25","down25"],
                                  ["isoversize75","up75"],["isoversize75","down75"]]:
        print(filtername, flagname)
        book_df_new = book_df1[book_df1[filtername] == flagname]
         #个数
        df_fnum = pd.DataFrame({flagname + "num":book_df_new .groupby("time_id")["seconds_in_bucket"].count()})
        data_merge_all =  pd.merge(data_merge_all, df_fnum, left_index=True, right_index=True,how="left")
    
    
        for roll_window in [5, 10]:
            #rolling指标
            price_name = "wap"
            roll_name0 = price_name + "roll_std" 
            roll_name = roll_name0 + str(roll_window)+"_"+flagname
            
            rolling_x = pd.DataFrame({roll_name:book_df_new.groupby("time_id")[price_name].rolling(roll_window).std()})
            rolling_x.reset_index(inplace=True)
            rolling_x.loc[:,"xpre"] = rolling_x.groupby("time_id")[roll_name].shift(1)
            #计算统计量因子
            data_merge = calc_rollingstats(rolling_x, roll_name)
            data_merge_all =  pd.merge(data_merge_all, data_merge, left_index=True, right_index=True,how="left")
        
        
        #全局做candle：wap，买盘，卖盘
        #candle因子
        price_name = "wap"
        vol_name = "vol_ab"
        amt_name = "amt_ab"
        df_data = cp.deepcopy(book_df_new)
        df_candle  = make_candle(df_data, price_name, vol_name, amt_name)
        
        list_save = [price_name + "candlef1", price_name + "candlef2", price_name + "candlef3",
                     price_name + "candlef41", price_name + "candlef42"]
       
        list_save = [i +"_"+flagname for i in list_save ]
        df_candle = cal_candlefactor(df_candle, price_name, vol_name, amt_name)
        
        col_orig = list(df_candle.columns)
        col_new = [i +"_"+flagname for i in col_orig ]
        df_candle.columns = col_new
        
        data_merge_all = pd.merge(data_merge_all, df_candle[list_save],left_index=True, right_index=True,how="left")

    #加filter做candle
    
#    切割，打flag，给权重， 加filter切历史，波动率，分买入卖出，大单小单，上行下行，主买主卖
#    全天上行波动率/全天波动率    
    #index 数据   #复杂    
    #calculate max and min bid-ask spread
    del data_merge_all["vol_orig"]
    
    return data_merge_all

In [19]:
book0 = util.load_book(0)
trade0 = util.load_trade(0)

In [20]:
temp = calculate_features2(book0,trade0)

isBS B
isBS S
isBS_big supB
isBS_big supS
isBS_big midBS
isoversize50 up50
isoversize50 down50
isoversize25 up25
isoversize25 down25
isoversize75 up75
isoversize75 down75


In [30]:
for i in temp.columns:
    print(i)

time_id
Bnum
waproll_std5_B_mean
waproll_std5_B_std
waproll_std5_B_skew
waproll_std5_B_autocorr
waproll_std10_B_mean
waproll_std10_B_std
waproll_std10_B_skew
waproll_std10_B_autocorr
wapcandlef1_B
wapcandlef2_B
wapcandlef3_B
wapcandlef41_B
wapcandlef42_B
Snum
waproll_std5_S_mean
waproll_std5_S_std
waproll_std5_S_skew
waproll_std5_S_autocorr
waproll_std10_S_mean
waproll_std10_S_std
waproll_std10_S_skew
waproll_std10_S_autocorr
wapcandlef1_S
wapcandlef2_S
wapcandlef3_S
wapcandlef41_S
wapcandlef42_S
supBnum
waproll_std5_supB_mean
waproll_std5_supB_std
waproll_std5_supB_skew
waproll_std5_supB_autocorr
waproll_std10_supB_mean
waproll_std10_supB_std
waproll_std10_supB_skew
waproll_std10_supB_autocorr
wapcandlef1_supB
wapcandlef2_supB
wapcandlef3_supB
wapcandlef41_supB
wapcandlef42_supB
supSnum
waproll_std5_supS_mean
waproll_std5_supS_std
waproll_std5_supS_skew
waproll_std5_supS_autocorr
waproll_std10_supS_mean
waproll_std10_supS_std
waproll_std10_supS_skew
waproll_std10_supS_autocorr
wapcand